<a href="https://colab.research.google.com/github/Niyayeshnazerian/ML-Project/blob/main/ML_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**فاز دوم**

In [1]:
!pip install mediapipe==0.10.13

In [2]:
import mediapipe as mp
print(mp.__version__)
print(hasattr(mp, "solutions"))

0.10.13
True


In [3]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

print("Pose loaded successfully ✅")

Pose loaded successfully ✅


In [4]:
!ls

sample_data


In [5]:
!pip install -q mediapipe opencv-python gradio numpy


In [6]:
import cv2
import mediapipe as mp
import numpy as np
import gradio as gr

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()


def tennis_talent_from_forehand(video_path):
    cap = cv2.VideoCapture(video_path)

    wrist_x = []
    shoulder_rotation = []
    valid_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)

        if result.pose_landmarks:
            lm = result.pose_landmarks.landmark

            rw = lm[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            rs = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            ls = lm[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            rh = lm[mp_pose.PoseLandmark.RIGHT_HIP.value]
            lh = lm[mp_pose.PoseLandmark.LEFT_HIP.value]

            wrist_x.append(rw.x)
            shoulder_rotation.append(abs(rs.x - ls.x) - abs(rh.x - lh.x))
            valid_frames += 1

    cap.release()

    if valid_frames < 20:
        return "❌ حرکت فورهند به‌درستی شناسایی نشد"

    # شاخص‌ها
    swing_range = max(wrist_x) - min(wrist_x)
    rotation = np.mean(shoulder_rotation)
    consistency = np.std(wrist_x)

    score = 0

    if swing_range > 0.35:
        score += 40
    if rotation > 0.05:
        score += 30
    if consistency < 0.12:
        score += 30

    if score >= 70:
        return f"🎾 استعداد مناسب برای تنیس ✅\nامتیاز: {score}/100"
    elif score >= 40:
        return f"⚠️ استعداد متوسط – نیاز به تمرین\nامتیاز: {score}/100"
    else:
        return f"❌ استعداد پایین برای تنیس\nامتیاز: {score}/100"


interface = gr.Interface(
    fn=tennis_talent_from_forehand,
    inputs=gr.Video(label="ویدیو اجرای فورهند"),
    outputs=gr.Textbox(label="نتیجه استعداد‌یابی"),
    title="استعداد‌یابی تنیس از روی فورهند",
    description="تحلیل دامنه حرکت، چرخش بدن و ثبات ضربه برای تشخیص استعداد تنیس"
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://12f61153ccb9417772.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
interface.launch(share=True, debug=True)
